Author: Max Boleininger

UK Atomic Energy Authority, 2022

# Processing threshold displacement energies for single-element materials 

In [4]:
import numpy as np
from scipy.spatial import cKDTree

from pathlib import Path
import sys  

# Get my_package directory path from Notebook
parent_dir = str(Path().resolve().parents[0])

# Add lib path to sys.path (needed to import the Lindhard model)
sys.path.insert(0, "%s/lib/" % parent_dir)

from lindhard import Lindhard

In [5]:
# import crystallographic point group transformations
fcctrans = np.loadtxt("crystal-symmetries/fcc_transformations.dat")
bcctrans = np.loadtxt("crystal-symmetries/bcc_transformations.dat")
hcptrans = np.loadtxt("crystal-symmetries/hcp_transformations.dat")

# reshape back to 3x3 transformation matrices
fcctrans = fcctrans.reshape(len(fcctrans),3,3)
bcctrans = bcctrans.reshape(len(bcctrans),3,3)
hcptrans = hcptrans.reshape(len(hcptrans),3,3)

In [6]:
def process_displacements(path, mass, znumber, stopping_cutoff, transformation, geo_freq=5):
    '''Import and process displacement events. '''

    # import geodesic sampling points
    # these coordinates are used for sampling E_d on the unit sphere
    # if request a value higher than 5, you will need to generate new coordinates. See geodesic-points/write_geo.py for this.
    kvec = np.loadtxt("./geodesic-points/geo.%d.dat" % geo_freq)
    
    # import displacement events, columns are:
    # energy | v_unit_x | v_unit_y | v_unit_z | nr_of_frenkel_pairs 
    pkadat = np.loadtxt(path)

    # only keep successful events (nr_of_frenkel_pairs>0)
    pkadat = pkadat[pkadat[:,-1]>=1,:-1]

    # convert recoil energy to damage energy
    tdmodel = Lindhard(mass, mass, znumber, znumber)

    # In the Lindhard stopping model, the charged particle loses energy continuously 
    # until the point of stopping. In our molecular dynamics simulations however, we use a 
    # typical electronic stopping model that only acts on particles with kinetic energy above e.g. 10 eV.
    # Therefore, the stopping model slightly underestimates the damage energy from simulation.
    # To correct for this, we add the energy loss for 10 eV particle back to the damage energy.
    dE  = stopping_cutoff - tdmodel(stopping_cutoff)
    pkadat[:,0] = tdmodel(pkadat[:,0]) + dE # set the 0th column to damage energy

    # compute all symmetrically equivalent points:
    # x' = M.x with some list concatenation (fast)
    nt = len(transformation)
    pkadatsymm = np.einsum('kij,lj->kli', transformation, pkadat[:,1:]).reshape(len(pkadat)*nt,3)
    pkadat = np.array([pkadat[:,0] for _i in range(nt)]).reshape(len(pkadat)*nt)
    
    # construct KDTree of geodesic dome vertices for quick neighbour search
    ktree = cKDTree(kvec)

    # for each randomly sampled kick, find nearest geodesic vertex 
    _,vertid = ktree.query(pkadatsymm)

    # create dictionary mapping sampled points to geodesic vertices
    cat = {}
    for i,v in enumerate(vertid):
        if v not in cat:
            cat[v] = [i]
        else:
            cat[v] += [i]

    # check if any vertex has no successful events. The fraction should be small (<0.001) with good sampling
    fr=len(np.setdiff1d([i for i in range(len(kvec))], list(cat.keys())))/len(kvec)
    print ("Fraction of unsampled vertices: %f" % fr)
    
    # compute mean and minimum Ed over geodesic vertices
    print ("Ed:     %8.4f eV" % np.mean([np.min(pkadat[cat[v]]) for v in cat]))
    print ("Ed,min: %8.4f eV" %  np.min([np.min(pkadat[cat[v]]) for v in cat]))

    datarr = []
    for v in range(len(kvec)):
        if v in cat:
            datarr += [[kvec[v][0], kvec[v][1], kvec[v][2], np.min(pkadat[cat[v]]), v]]
        else:
            datarr += [[kvec[v][0], kvec[v][1], kvec[v][2], 0, v]]

    # data array contains bin (geodesic coordinate), minimum threshold energy in this bin, and number of counts in this bin 
    return datarr


In [13]:
# process edsample data
mass = 58.6934          # atomic mass (Dalton) for Nickel
znumber = 28            # Z number for Nickel
stopping_cutoff = 10.0  # electronic stopping cutoff (eV) used in the edsample simulation

# Enter here the path of the log file generated with geted.py
path = './example-data/ni_edsample.log'
datarr = process_displacements(path, mass, znumber, stopping_cutoff, fcctrans)

Fraction of unsampled vertices: 0.000000
Ed:      36.0236 eV
Ed,min:  21.6994 eV
